# Classificação de Discurso de Ódio 
-------------------------------------
## Abordagem: tf-idf com Regressão Logística

-------------------------------------

## Importações e Definições

In [2]:
import pandas as pd
import numpy as np
from datasets import load_dataset

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder

from lime.lime_text import LimeTextExplainer

from utils import preprocess_text, format_lime_output, print_multilabel_metrics

[nltk_data] Downloading package stopwords to /home/penido/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
target = ['aggressive', 'hate', 'ageism', 'aporophobia', 'body_shame', 'capacitism', 'lgbtphobia', 'political', 'racism', 'religious_intolerance', 'misogyny', 'xenophobia', 'other']
features = 'text'

In [4]:
# Configurações para o pré-processamento
config = {
    "lowercase": True,
    "remove_accents": True,
    "remove_punctuation": True,
    "remove_numbers": True,
    "remove_urls": True,
    "remove_mentions_hashtags": True,
    "expand_abbreviations": True,
    "expand_contractions": False,
    "normalize_laughter": True,
    "remove_emojis": True,
    "remove_stopwords": True,
    "lemmatize": True,
    "stemming": False,
    "pos_filter": False,
    "min_token_length": 2,
    "negation_scope": False,
    "replace_swears": False,
    "split_hashtags": False,
    "merge_mwes" : True,
    "replace_named_entities" : False
}

le = LabelEncoder()

--------------------------------------------
## Prepara o Conjunto de Dados

In [5]:
# 1. Carrega o dataset TuPyE multilabel
df = load_dataset("Silly-Machine/TuPyE-Dataset", name="multilabel")

train_df = df['train'].to_pandas()
test_df = df['test'].to_pandas()

X_train_raw = train_df[features]
train_df["label_comb"] = train_df[target].astype(str).agg("".join, axis=1)
train_df["class_id"] = le.fit_transform(train_df["label_comb"])
y_train = train_df["class_id"].values

X_test_raw = test_df[features]
y_test = test_df[target].values

In [6]:
train_df["class_id"].value_counts(normalize=True)

class_id
0     0.709051
7     0.112297
13    0.036927
1     0.028912
8     0.022156
        ...   
56    0.000029
61    0.000029
91    0.000029
6     0.000029
53    0.000029
Name: proportion, Length: 96, dtype: float64

### Aplica Pré-Processamento

In [7]:
X_train = X_train_raw.apply(lambda x: preprocess_text(x, config))
X_test = X_test_raw.apply(lambda x: preprocess_text(x, config))

--------------------------------------------
## Treinamento do Modelo

In [8]:
tfidf = TfidfVectorizer(
    max_features=10000,       
    ngram_range=(1, 4),       
    min_df=2,                
    max_df=0.95,             
    sublinear_tf=True        
)

In [9]:
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

In [10]:
model = LogisticRegression(max_iter=10000)
model.fit(X_train_tfidf, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,10000
,multi_class,'deprecated'


-----------------------------
## Define Explicador - LIME

In [11]:
def predict_proba(text_list):
    X_test = tfidf.transform(text_list)
    return model.predict_proba(X_test)

In [12]:
explainer = LimeTextExplainer(class_names=target)

### Teste com uma amostra

In [13]:
_texto = "vai tomar no seu cu seu viadinho de merda"
texto = preprocess_text(_texto, config)
exp = explainer.explain_instance(texto, predict_proba, num_features=6)

In [14]:
format_lime_output(texto, predict_proba, target, exp)

Categorias de discurso de ódio:
O texto não é discurso de ódio 🕊️


-----------------------------------
## Avaliação no conjunto de Teste

In [15]:
y_pred = model.predict(X_test_tfidf)

# converte saida inteira em multilabel de volta
multilabel_pred = [list(map(int, s)) for s in le.inverse_transform(y_pred)]

In [16]:
print_multilabel_metrics(y_test, multilabel_pred)


📊 Avaliação Multilabel
✔️ F1 Score (Micro):     0.3626
✔️ F1 Score (Macro):     0.1864
✔️ F1 Score (Weighted):  0.3523
⚠️ Hamming Loss:         0.0380
✅ Subset Accuracy:      0.7493


-----------------------------------
## Salva explicações

In [17]:
X_test

0          dizer atitude ministro cujo pasta dever con...
1                             Fernando haddad representar
2                                branco vergonha raca pqp
3                             todo mes pagar querer porra
4                                     menina insuportavel
                              ...                        
8729                       rt     numero tiro ir levar cu
8730                                         viar certeza
8731                                         falar porrar
8732    Paulo freire tao bom nenhum pais mundo copiar ...
8733    ir ficar feio reforma previdencia passar vez a...
Name: text, Length: 8734, dtype: object

In [19]:
from tqdm import tqdm

def extrair_top_palavras_lime(texto, explainer, predict_proba, top_k=3, score_min=0.1):
    explicacao = explainer.explain_instance(texto, predict_proba, num_features=20)
    
    # Pega os pares (palavra, score), filtra por score mínimo, e ordena
    palavras_filtradas = [
        (palavra, score)
        for palavra, score in explicacao.as_list()
        if abs(score) >= score_min
    ]
    
    # Ordena por importância (absoluta) e pega as top-k palavras
    palavras_topk = sorted(palavras_filtradas, key=lambda x: abs(x[1]), reverse=True)[:top_k]
    
    # Só retorna a palavra (sem o score)
    return [palavra for palavra, _ in palavras_topk]


In [21]:
tqdm.pandas()  # para ver progresso

test_df["palavras_lime"] = X_test.progress_apply(
    lambda x: extrair_top_palavras_lime(x, explainer, predict_proba)
)

  0%|          | 14/8734 [00:27<4:44:47,  1.96s/it]


KeyboardInterrupt: 